In [0]:
#Imports
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f

#Dataset Schema
schema = StructType([
  StructField("order_id", StringType()),
  StructField("timestamp", StringType()),
  StructField("customer_id", StringType()),
  StructField("product_id", StringType()),
  StructField("category", StringType()),
  StructField("price", DoubleType()),
  StructField("quantity", IntegerType()),
  StructField("total_amount", DoubleType()),
  StructField("city", StringType()),
  StructField("state", StringType()),
  StructField("country", StringType()),
  StructField("latitude", StringType()),
  StructField("longitude", StringType()),
  StructField("delivery_status", StringType())])

  # Azure Event Hub Configuration
event_hub_namespace='mn-ecommerce-namespace'
event_hub_name='ecommerce-orders'
event_hub_conn_str='Endpoint=sb://mn-ecommerce-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=<shared_access_key>'

eh_conf = {
    'kafka.bootstrap.servers':f'{event_hub_namespace}.servicebus.windows.net:9093',
      'subscribe': event_hub_name,
          'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
          'kafka.sasl.mechanism': 'PLAIN',
          'kafka.security.protocol':  'SASL_SSL',
          'startingOffsets': 'latest',
          'failOnDataLoss': 'false'
}

#Stream from Event hub
df_raw = (
    spark.readStream
    .format("kafka")
    .options(**eh_conf)
    .load()
)

#Stream read is in bytes so we need to convert it to string
#Parse json from kafka stream
df_orders = (
    df_raw.selectExpr('Cast(value AS STRING) as json')
    .select(from_json("json", schema).alias("data"))
    .select("data.*")
)

#access the bronze table 
#we can use either key or string based access to accees the azure storage account (Using key based access)
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
  "<password"
)

#specifying path to write data
bronze_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/bronze"

(
    df_orders.writeStream
    .format('delta')
    .outputMode("append")
    .option("checkpointLocation",bronze_path+"/_checkpoint")
    .start(bronze_path)
)

In [0]:
df_bronze = spark.read.format('delta').load(bronze_path)
display(df_bronze)

order_id,timestamp,customer_id,product_id,category,price,quantity,total_amount,city,state,country,latitude,longitude,delivery_status
1c0c43d1-3586-404e-b1d3-2bb92ad1ef44,2025-11-24T11:44:54.637017,2ef73152-8200-401f-b6e7-6c735d7a95b2,8e0cbbe8-1ebe-41d3-83fc-d1a43a539f01,Electronics,929.94,4,3719.76,Los Angeles,CA,USA,34.0522,-118.2437,Shipped
a35e30fd-2c42-4b9a-a85a-0977e49f01a2,2025-11-24T11:44:56.639811,d180edc6-17d1-4943-8820-ac3cf8e852eb,0a2970d8-f9ed-4995-944d-93363d79cde6,Electronics,792.73,4,3170.92,Chicago,IL,USA,41.8781,-87.6298,Processing
0c906345-4dd5-4663-a1e4-2b11c70b17b0,2025-11-24T11:44:58.641452,7d4c4f15-7a59-46be-92a8-b9a412b305da,11940d86-9c43-4d03-a4c9-6a5bfdb08567,Electronics,558.41,5,2792.05,Los Angeles,CA,USA,34.0522,-118.2437,Cancelled
46d14685-03e6-45ef-9631-d5fe8f787256,2025-11-24T11:45:00.643702,7d2868a6-29a4-4faf-a587-42ab29c534d6,e83c608f-7fa2-4091-a5cb-b8bf57ddf943,Toys,1026.58,5,5132.9,Houston,TX,USA,29.7604,-95.3698,Processing
c76b5d17-5fca-4d61-8450-fd61fadd4f13,2025-11-24T11:45:02.646477,e61a3275-aa4f-4a8e-b35c-f08d1083411c,61e79833-8c57-4301-ae6e-8c7b98978eca,Clothing,680.79,4,2723.16,Houston,TX,USA,29.7604,-95.3698,Shipped
682d840e-a60f-481f-91f9-e040dd27f475,2025-11-24T11:45:04.648704,89a581b7-e50d-442f-8a48-8705d2d9a1da,7caa5041-fa59-4a3a-8bdf-2b912f730fac,Toys,861.04,5,4305.2,Chicago,IL,USA,41.8781,-87.6298,Delivered
98fb8d2e-931f-47bc-a686-866d7791df05,2025-11-24T11:45:06.653728,2f447750-9847-4189-9335-cfe7a5d430b6,515ca3a9-22df-4eeb-bf68-ff562bc52017,Electronics,1845.74,5,9228.7,New York,NY,USA,40.7128,-74.006,Processing
041e8392-e831-4d6e-b66f-dee898af917d,2025-11-24T11:45:08.656069,056c4c6d-0975-4963-9e79-086607885003,5c5c9d6b-7a57-4429-9464-c8f3dcd2f430,Home Decor,502.55,5,2512.75,Los Angeles,CA,USA,34.0522,-118.2437,Processing
78046009-ef87-443c-bccf-228b5635a2ec,2025-11-24T11:45:10.657304,7a9e8749-836e-4eec-ab7b-6923adbba2d3,f132109f-97d4-41dc-9766-db78c2d4d3b9,Home Decor,276.17,1,276.17,Los Angeles,CA,USA,34.0522,-118.2437,Shipped
91ba8249-71ec-45b2-865b-2f934f75cb44,2025-11-24T11:45:12.659985,80d786da-315c-4a9c-8931-d4a58201981f,a2ae9df8-6aa0-4039-848f-39e33e95af6e,Books,1709.59,4,6838.36,New York,NY,USA,40.7128,-74.006,Delivered
